**Parameterizing small molecule for gromacs/Amber simulations**

first draw your molecule, and minimize it with MM level. We want to calculate resp charges which will be used for the MD simulation. The gaussian input .com file can be created using antechamber via the following command. This tutorial is modified version of https://ambermd.org/tutorials/basic/tutorial0/

In [ ]:
antechamber -i name.pdb -fi pdb -o *.com -fo gcrt

This will print .com file with title #HF/6-31G* SCF=tight Test Pop=MK iop(6/33=2) iop(6/42=6). Now you need to run guassian program in order to get .log file, which will be used in the next step

In [ ]:
antechamber -i *.log -fi gout -o *.mol2 -fo mol2 -c resp


*#you can calculate resp charges by Using Multiwfn (http://sobereva.com/multiwfn) to calculate RESP charges. The procedure is much easier and the functionality is more powerful than Antechamber, and you do not need to boring with connectivity. Please follow the examples in Section 4.7.7 of Multiwfn manual.*

In [ ]:
$tleap
Source leaprc.protein.ff14SB
Source leaprc.water.opc 
Source leaprc.protein.ff14SB 
Source leaprc.water.opc
Source leaprc.gaff2
ligand = loadmol2 genistein_final.mol2 
check ligand
loadamberparams genistein_final.mfrcmod
saveoff ligand ligand.lib 
setbox ligand vdw 6
#solvatebox ligand TIP3PBOX 10.0
saveamberparm ligand ligand.inpcrd ligand.prmptop
quit


you will get final ligand in water output with the paramters compatible for amber simulation. Note that solvatebox is additional command if you want to add water molecule, however if you just interested ligand parameterization and interested in gromacs topology, you can skip this step.

In case you want to visualize the output file in .pdb format, you can use following command

In [ ]:
ambpdb -p ligand.inpcrd -c ligand.prmptop > ligand.pdb

gromacs to amber topology can be converted via python module called parmed. make a script.py file via the following input

In [ ]:
import parmed as pmd
parm = pmd.load_file('ligand.prmptop','ligand.inpcrd')
parm.save('ligand.itp',format='gromacs')
parm.save('ligand.gro')
parm.save('ligand.top') #,format='gromacs')

In [ ]:
python script.py

You will get gromacs input files and you are good to go with gromacs simulations. In case if want to simulate ligand in water, i am here giving you .mdp files [em.mdp](https://github.com/athar91/small_molecule_paramterization/blob/main/em.mdp), [eq.mdp](https://github.com/athar91/small_molecule_paramterization/blob/main/eq.mdp), [run.mdp](https://github.com/athar91/small_molecule_paramterization/blob/main/run.mdp)), sample .top file ( [system_EMPTY.top](https://github.com/athar91/small_molecule_paramterization/blob/main/system_EMPTY.top)), and you just need to execute bash script ( [prepare_1mol_AA_system.sh](https://github.com/athar91/small_molecule_paramterization/blob/main/prepare_1mol_AA_system.sh/)), it will simply take care of everything.

enter the ligand name in .top file

In [ ]:
./prepare_1mol_AA_system.sh  ligand.gro  spc216.gro  SOL 3

You might need to apply position restraint on the ligand, if its a complex/dimer/trimer. CHECK THIS http://www.mdtutorials.com/gmx/complex/06_equil.html

In [ ]:
gmx make_ndx -f ligand.gro -o index_ligand.ndx
...
 > 0 & ! a H*
 > q

In [ ]:
gmx genrestr -f ligand.gro -n index_ligand.ndx -o posre_ligand.itp -fc 1000 1000 1000

edit .top file (include posres) and define posres in .mdp file [define = -DPOSRES -DPOSRES_LIG]

In [ ]:
; Ligand position restraints
#ifdef POSRES_LIG
#include "posre_ligand.itp"
#endif


*However*, if you want to simulate using sander, you can use following command

In [ ]:
sander -O -i 01_Min.in -o 01_Min.out -p ligand.pmptop -c ligand.inpcrd -r 01_Min.ncrst -inf 01_Min.mdinfo

**Note that in case of dimeric complex, you might need to define .itp file separately for the two unit via these steps:**

1.  generate a complex.pdb file
2.  grep -v MOL complex.pdb > onlycalix.pdb
3. grep MOL complex.pdb > onlygenistein.pdb
4. acpype (https://www.bio2byte.be/acpype) of both monomers (calix and genistein) using pdb file to generate parameters
5. download gromacs file from acpype
6. combine two .gro monomer files(from acpype) files to make final dimer (complex.gro)
7. Make one atomtype file, add all atom into one. caliix atom type into one file (genistein.itp). Genistein will be defined first in .top file.
8. run prep* bash script (generate iitial solvated file) followed by energy minmization.
9. gmx_mpi make_ndx -f 1-min.gro -o index.ndx
10. gmx_mpi genrestr -f genonlycalix_GMX.gro -n index.ndx -o posre_onlycalix.itp -fc 1000 1000 1000
11. gmx_mpi genrestr -f onlycalix_GMX.gro -n index.ndx -o posre_onlygenistein.itp -fc 1000 1000 1000
12. define coupling groups and DPOSRES

